# American Option Pricing with Longstaff-Schwartz

This notebook demonstrates pricing **American options** using the **Longstaff-Schwartz Monte Carlo** algorithm - a classic interview topic in quantitative finance.

## What You'll See
1. Compare American vs European put prices
2. Visualize the early exercise premium
3. Explore how parameters affect the American premium
4. Understand the LSM regression approach

In [ ]:
using Pkg
Pkg.activate("..")

using Quasar
using Plots
using Statistics
gr()

## 1. American vs European Put

American options can be exercised at any time before expiry. For puts, this early exercise feature has value - especially for deep in-the-money options.

In [ ]:
# Market parameters
S0 = 100.0    # Spot price
K = 100.0     # Strike (ATM)
T = 1.0       # 1 year to expiry
r = 0.05      # Risk-free rate
sigma = 0.2   # Volatility

dynamics = GBMDynamics(r, sigma)

# Price European put (closed-form via Monte Carlo)
println("Pricing options with 50,000 paths...")
@time eu_put = mc_price(S0, T, EuropeanPut(K), dynamics; npaths=50000)
@time am_put = lsm_price(S0, T, AmericanPut(K), dynamics; npaths=50000, nsteps=50)

# Also compute Black-Scholes for reference
bs_put = black_scholes(S0, K, T, r, sigma, :put)

println("\n" * "="^50)
println("ATM Put Prices (S0 = K = $K)")
println("="^50)
println("Black-Scholes (European): \$$(round(bs_put, digits=4))")
println("Monte Carlo (European):   \$$(round(eu_put.price, digits=4)) ± $(round(eu_put.stderr, digits=4))")
println("LSM (American):           \$$(round(am_put.price, digits=4)) ± $(round(am_put.stderr, digits=4))")
println("\nEarly Exercise Premium:   \$$(round(am_put.price - eu_put.price, digits=4))")
println("Premium as % of European: $(round((am_put.price/eu_put.price - 1)*100, digits=2))%")

## 2. Early Exercise Premium Across Strikes

The early exercise premium is largest for **deep in-the-money** puts. Why? Because you can exercise, receive K, and invest at the risk-free rate.

In [ ]:
# Compute prices across strikes
strikes = collect(70.0:5.0:130.0)

eu_prices = Float64[]
am_prices = Float64[]
eu_se = Float64[]
am_se = Float64[]

println("Pricing across strikes...")
for K in strikes
    eu = mc_price(S0, T, EuropeanPut(K), dynamics; npaths=30000)
    am = lsm_price(S0, T, AmericanPut(K), dynamics; npaths=30000, nsteps=50)
    push!(eu_prices, eu.price)
    push!(am_prices, am.price)
    push!(eu_se, eu.stderr)
    push!(am_se, am.stderr)
    print(".")
end
println(" done!")

In [ ]:
# Plot American vs European prices
p1 = plot(strikes, eu_prices,
          ribbon=1.96 .* eu_se,
          label="European Put",
          xlabel="Strike",
          ylabel="Price (\$)",
          title="American vs European Put Prices",
          linewidth=2,
          fillalpha=0.2)

plot!(p1, strikes, am_prices,
      ribbon=1.96 .* am_se,
      label="American Put",
      linewidth=2,
      fillalpha=0.2)

# Add intrinsic value line
intrinsic = max.(strikes .- S0, 0.0)
plot!(p1, strikes, intrinsic,
      label="Intrinsic Value",
      linestyle=:dash,
      color=:gray,
      linewidth=1)

vline!(p1, [S0], label="Spot", linestyle=:dot, color=:black)
p1

In [ ]:
# Plot the early exercise premium
premium = am_prices .- eu_prices
premium_pct = (am_prices ./ eu_prices .- 1) .* 100

p2 = plot(strikes, premium,
          label="Early Exercise Premium (\$)",
          xlabel="Strike",
          ylabel="Premium",
          title="Early Exercise Premium",
          linewidth=2,
          color=:green,
          fill=0,
          fillalpha=0.3)

vline!(p2, [S0], label="Spot", linestyle=:dot, color=:black)
p2

**Key Observation**: The early exercise premium increases as the put goes deeper in-the-money (higher strikes relative to spot). This is because exercising a deep ITM put lets you receive the strike price immediately and invest it at the risk-free rate.

## 3. Effect of Interest Rates

Higher interest rates make early exercise more attractive for puts (invest the proceeds) but less attractive for calls.

In [ ]:
# Compare early exercise premium for different rates
rates = [0.01, 0.03, 0.05, 0.08, 0.10]
K_test = 110.0  # ITM put

premiums = Float64[]

for r in rates
    dyn = GBMDynamics(r, sigma)
    eu = mc_price(S0, T, EuropeanPut(K_test), dyn; npaths=20000)
    am = lsm_price(S0, T, AmericanPut(K_test), dyn; npaths=20000, nsteps=50)
    push!(premiums, am.price - eu.price)
end

bar(string.(Int.(rates .* 100)) .* "%", premiums,
    xlabel="Interest Rate",
    ylabel="Early Exercise Premium (\$)",
    title="Premium vs Interest Rate (K=$K_test, S0=$S0)",
    legend=false,
    color=:steelblue)

## 4. Effect of Time to Expiry

Longer-dated options have more opportunities for early exercise.

In [ ]:
# Compare across maturities
maturities = [0.25, 0.5, 1.0, 2.0, 3.0]
K_test = 110.0

eu_by_T = Float64[]
am_by_T = Float64[]

for T in maturities
    nsteps = max(Int(round(T * 50)), 10)
    eu = mc_price(S0, T, EuropeanPut(K_test), dynamics; npaths=20000)
    am = lsm_price(S0, T, AmericanPut(K_test), dynamics; npaths=20000, nsteps=nsteps)
    push!(eu_by_T, eu.price)
    push!(am_by_T, am.price)
end

groupedbar([eu_by_T am_by_T],
           bar_position=:dodge,
           label=["European" "American"],
           xticks=(1:length(maturities), string.(maturities) .* "Y"),
           xlabel="Time to Expiry",
           ylabel="Price (\$)",
           title="Put Prices by Maturity (K=$K_test)")

## 5. American Calls (No Dividends)

For **non-dividend paying stocks**, American calls should **never** be exercised early. The American call = European call.

In [ ]:
# American call vs European call
eu_call = mc_price(S0, T, EuropeanCall(K), dynamics; npaths=50000)
am_call = lsm_price(S0, T, AmericanCall(K), dynamics; npaths=50000, nsteps=50)

println("ATM Call Prices (no dividends):")
println("  European: \$$(round(eu_call.price, digits=4)) ± $(round(eu_call.stderr, digits=4))")
println("  American: \$$(round(am_call.price, digits=4)) ± $(round(am_call.stderr, digits=4))")
println("  Difference: \$$(round(am_call.price - eu_call.price, digits=4))")
println("\nAs expected, American call ≈ European call (no early exercise premium)")

## 6. Understanding the LSM Algorithm

The Longstaff-Schwartz algorithm works by:
1. **Simulating paths forward** from today to expiry
2. **Working backwards** through time
3. At each step, **regressing** the discounted future cashflow on polynomial basis functions of the current spot
4. **Comparing** immediate exercise value vs. estimated continuation value

Let's visualize the continuation value regression at one time step.

In [ ]:
# Simulate paths to visualize LSM regression
using Random
Random.seed!(42)

# Generate paths
npaths = 5000
nsteps = 20
T = 1.0
K = 100.0
dt = T / nsteps

paths = zeros(npaths, nsteps + 1)
paths[:, 1] .= S0

for i in 1:npaths
    for t in 1:nsteps
        Z = randn()
        drift = (r - 0.5 * sigma^2) * dt
        diffusion = sigma * sqrt(dt) * Z
        paths[i, t+1] = paths[i, t] * exp(drift + diffusion)
    end
end

# At time step 10 (mid-life), look at ITM paths
t_idx = 10
S_t = paths[:, t_idx + 1]

# Intrinsic value at this time
intrinsic_t = max.(K .- S_t, 0.0)
itm = intrinsic_t .> 0

# "Continuation value" = discounted terminal payoff (simplified)
terminal_payoff = max.(K .- paths[:, end], 0.0)
discount = exp(-r * (T - t_idx * dt))
continuation = discount .* terminal_payoff

# Fit regression on ITM paths
S_itm = S_t[itm]
C_itm = continuation[itm]

# Polynomial regression: C = β0 + β1*S + β2*S²
X = hcat(ones(length(S_itm)), S_itm, S_itm.^2)
β = X \ C_itm

# Plot
scatter(S_itm, C_itm,
        label="Discounted Future Cashflow",
        xlabel="Spot at t=$(t_idx*dt) years",
        ylabel="Value",
        title="LSM Regression: Continuation Value",
        alpha=0.3,
        markersize=2)

# Fitted continuation curve
S_range = range(minimum(S_itm), maximum(S_itm), length=100)
C_fitted = β[1] .+ β[2] .* S_range .+ β[3] .* S_range.^2
plot!(S_range, C_fitted, label="Fitted Continuation", linewidth=3, color=:red)

# Intrinsic value
plot!(S_range, max.(K .- S_range, 0), label="Intrinsic Value", linewidth=2, linestyle=:dash, color=:green)

**Reading the Plot:**
- Each blue dot is a simulated path at time t
- The red curve is the **fitted continuation value** (expected value of holding)
- The green dashed line is the **intrinsic value** (value of exercising now)
- **Exercise when intrinsic > continuation** (green above red)

## 7. Convergence with Number of Paths

LSM accuracy improves with more Monte Carlo paths.

In [ ]:
# Convergence study
path_counts = [1000, 2000, 5000, 10000, 20000, 50000]
prices = Float64[]
stderrs = Float64[]

K = 110.0  # ITM put

for n in path_counts
    result = lsm_price(S0, T, AmericanPut(K), dynamics; npaths=n, nsteps=50)
    push!(prices, result.price)
    push!(stderrs, result.stderr)
end

plot(path_counts, prices,
     ribbon=1.96 .* stderrs,
     label="LSM Price ± 95% CI",
     xlabel="Number of Paths",
     ylabel="Price (\$)",
     title="LSM Convergence",
     linewidth=2,
     xscale=:log10,
     marker=:circle)

## Summary

This notebook demonstrated:

1. **American vs European Options** - American puts have an early exercise premium
2. **Longstaff-Schwartz Algorithm** - Regression-based Monte Carlo for American options
3. **Key Drivers** - Premium increases with: higher rates, longer maturity, deeper ITM
4. **American Calls** - No early exercise premium for non-dividend stocks

### Interview Talking Points
- LSM uses **backward induction** with **regression** to estimate continuation values
- The regression is on **in-the-money paths only** (others have zero value)
- Polynomial basis functions (1, S, S²) are simple but effective
- LSM is **low-biased** (underestimates true price) due to using same paths for regression and exercise decision